# Tarea 2: Procesamiento de Lenguaje Natural



Integrantes:

* Sebastián Alday (Rut: 18294095-K)
* Paula Canales G. (Rut: 18845155-1)
* Álvaro Neira R. (Rut: 13757209-5)
* Matias Rodriguez U. (Rut: 18362815-1)

# Introducción

# 1. Creacion y limpieza de la base de datos 

## 1.1 Elija una organización que cuenta con una lista de preguntas y respuestas

Chilexpress
https://centrodeayuda.chilexpress.cl/home

## 1.2 Genere una tabla (tablaQA.xls) con las preguntas y respuestas de la organización seleccionada.

In [ ]:
!wget https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv

--2021-12-23 18:18:51--  https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9zty4fuh8boofda/tablaQA.csv [following]
--2021-12-23 18:18:51--  https://www.dropbox.com/s/raw/9zty4fuh8boofda/tablaQA.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc13f90d26f02375e179493a0549.dl.dropboxusercontent.com/cd/0/inline/BcYWFVkC5MGooCW7pY-mKRjIlnEyn7qfLGWnA1kPCeQj-2iBxOHR1BRiTuurc0n9Lgz6wSH8ENolBqhFUQ9sml4d5hZ-HTZsCVLRj2MWpA5TK6KkE5lsgaDC7VINOZ26pAVucv-Ex9qLh9N7oq9MrqLC/file# [following]
--2021-12-23 18:18:51--  https://uc13f90d26f02375e179493a0549.dl.dropboxusercontent.com/cd/0/inline/BcYWFVkC5MGooCW7pY-mKRjIlnEyn7qfLGWnA1kPCeQj-2iBxOHR1BRiTuurc0n9Lgz6wSH8ENolBqhFUQ9sml4d5h

In [ ]:
import pandas as pd
qa = pd.read_csv('/content/tablaQA.csv', sep=';')
qa.head()

,Pregunta,Respuesta
0,¿Qué es la Orden de Transporte (OT)?,La Orden de Transporte (OT) es el número asoci...
1,¿Cuál es el plazo para retirar un envío desde ...,El plazo para retirar un envío desde una sucur...
2,¿Puede otra persona retirar mi envío de una su...,Para que una persona que no es el destinatario...
3,¿Se puede modificar la dirección de envío?,"Sí, el cambio puede ser solicitado por el remi..."
4,¿Puedo cambiar el destino del envío desde una ...,"Sí, el cambio puede ser solicitado por el remi..."


In [ ]:
qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pregunta   46 non-null     object
 1   Respuesta  46 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


## 1.3 Genere una tabla (tiposmensajes.xls) con ejemplos de las siguientes clases de mensaje: "saludo", "despedida", "nombre", "informacion"

In [ ]:
!wget https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv

--2021-12-23 18:18:52--  https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv [following]
--2021-12-23 18:18:53--  https://www.dropbox.com/s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc961ebff776628e6e00fb3cb1cc.dl.dropboxusercontent.com/cd/0/inline/BcZoAaLK2I9FXl_ysuSgTeeBqJRCjqOMkpVoKm8e_TKNQtbjaDQwEiF7pikI_lntHWbE2J_GPX2XVshP8CFiIN8GKiDv-7l4D3Bx52hc63RC8Ik9uyFZYrC001psT09fC7R2iGQL-hjFUsxkl1QVfyJ6/file# [following]
--2021-12-23 18:18:53--  https://uc961ebff776628e6e00fb3cb1cc.dl.dropboxusercontent.com/cd/0/inline/BcZoAaLK2I9FXl_ysuSgTeeBqJRCjqOMkpVoKm8e_TKNQtbjaDQwEiF7pikI_lntHWbE2J_G

In [ ]:
messages = pd.read_csv('/content/tiposmensajes.csv', sep=',')
messages.head()

,Clase,Mensaje
0,saludo,Hola
1,saludo,Hi
2,saludo,Buenos días
3,saludo,Buenas tardes
4,saludo,Como estas


## 1.4 Genere respuestas predeterminadas (respuestasDefecto.xls) para los mensajes de tipo "saludo", "despedida", "nombre"

In [ ]:
!wget https://www.dropbox.com/s/kukxrqpjxi0l4ei/respuestasDefecto.csv

--2021-12-23 18:18:54--  https://www.dropbox.com/s/kukxrqpjxi0l4ei/respuestasDefecto.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kukxrqpjxi0l4ei/respuestasDefecto.csv [following]
--2021-12-23 18:18:54--  https://www.dropbox.com/s/raw/kukxrqpjxi0l4ei/respuestasDefecto.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb8f254a0d7e2006887df3cf304.dl.dropboxusercontent.com/cd/0/inline/Bcaz42aJqy5WJ1aW5eOkhk5g7oaPf_fF9FyiTTwBIIbFz8M0z5YRsZ88FbKcsJkNxI0OT-sVtu-A324Lgp6c9PgV3RKJRlzH3TTk1hr74m7py628a2t0TqLLTv5wubnzajFasBzk8zoAyj2CWkkPKhV3/file# [following]
--2021-12-23 18:18:54--  https://ucb8f254a0d7e2006887df3cf304.dl.dropboxusercontent.com/cd/0/inline/Bcaz42aJqy5WJ1aW5eOkhk5g7oaPf_fF9FyiTTwBIIbFz8M0z5YRsZ88FbKc

In [ ]:
answers = pd.read_csv('/content/respuestasDefecto.csv', sep=',')
answers.head()

,Clase,Mensaje
0,saludo,"Hola, en qué puedo ayudarte?"
1,saludo,"Hola, cuentame qué necetias"
2,saludo,"Buenos días, en qué puedo ayudarte?"
3,saludo,"Buenas tardes, en qué puedo ayudarte?"
4,saludo,"Un gusto saludar, en qué puedo ayudarte?"


## 1.5 Describa en términos generales las tablas que construyó.



*   Tabla QA: Se construyó en base a las preguntas y respuestas del centro de ayuda de Chilexpress https://centrodeayuda.chilexpress.cl/home . Se extrajo cada pregunta y respuesta de cada categoría (Envíos Nacionales, Envíos y retiros en App, Envíos internacionales, Giros en Chile y el mundo, Atención en sucursal, Cuenta empresa) y se eliminaron las preguntas repetidas. 
*   Tipos de mensajes: se construyó con mensajes tipo de saludo, despedida y nombre. Para la información se utilizó las preguntas obtenidas en la Tabla QA.
*   Respuestas por defecto: se construyó con respuestas tipo de saludo, despedida y nombre de nuestro bot Eva



# 2. Análisis de distancia

Para el primer chatbot utilizará una métrica de su elección para responder
a cada pregunta/texto del usuario.

## 2.1 Utilice algún embeddings utilizado en el curso para codificar el texto de entrada

### 2.1.1 Word2Vec

In [ ]:
import nltk
import gensim
import re
import pandas as pd
import sklearn
import numpy as np

import sklearn.linear_model
import sklearn.model_selection

In [ ]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

def preprocessor(text):
  text = normalizer(text)
  tokens = nltk.tokenize.casual_tokenize(text)
  return tokens

def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.nan_to_num(np.mean(vectors,axis=0)))

In [ ]:
corpus = messages.Mensaje.tolist()
corpus_preprocessed = list(map(preprocessor,corpus))
model = gensim.models.word2vec.Word2Vec(sentences = corpus_preprocessed)
model

### 2.1.2 ELMO

In [ ]:
!pip install overrides==3.1.0
!pip install elmoformanylangs

  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10186 sha256=fe10a8608a01201f0cb6320f59ad190c1720ad597f3ac163801ece9bd6695888
  Stored in directory: /root/.cache/pip/wheels/3a/0d/38/01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
Successfully built overrides
     |████████████████████████████████| 42 kB 649 kB/s 


In [ ]:
!wget http://vectors.nlpl.eu/repository/11/145.zip
!unzip 145.zip

--2021-12-23 18:19:31--  http://vectors.nlpl.eu/repository/11/145.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 419687058 (400M) [application/zip]
Saving to: ‘145.zip’

145.zip             100%[===================>] 400.24M  10.2MB/s    in 42s     

2021-12-23 18:20:14 (9.60 MB/s) - ‘145.zip’ saved [419687058/419687058]

Archive:  145.zip
  inflating: char.dic                
  inflating: config.json             
  inflating: encoder.pkl             
  inflating: meta.json               
  inflating: README                  
  inflating: token_embedder.pkl      
  inflating: word.dic                


In [ ]:
import elmoformanylangs

In [ ]:
e = elmoformanylangs.Embedder('')

2021-12-23 18:20:30,802 WARNING: Could not find config.  Trying cnn_50_100_512_4096_sample.json
2021-12-23 18:20:30,806 WARNING: Could not find config.  Trying /usr/local/lib/python3.7/dist-packages/elmoformanylangs/configs/cnn_50_100_512_4096_sample.json
2021-12-23 18:20:30,916 INFO: char embedding size: 2637
2021-12-23 18:20:31,965 INFO: word embedding size: 185214
2021-12-23 18:20:47,182 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(185214, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2637, 50, padding_idx=2634)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel

In [ ]:
sentences = list(map(preprocessor,messages.Mensaje))
features_raw = e.sents2elmo(sentences)

2021-12-23 18:43:55,809 INFO: 1 batches, avg len: 10.8


## 2.2 Con el texto de entrada codificado y usando embeddings proponga una manera de identificar la clase del texto de entrada.

In [ ]:
def predict_class(question):
  tokens = preprocessor(question)
  vector = vectorizer(tokens,model).reshape(1, -1)
  return clf.predict(vector)

### 2.2.1 Word2Vec

In [ ]:
features = np.zeros(shape=(len(messages),model.wv.vectors.shape[1]))
for i,msg in enumerate(messages.Mensaje):
  features[i,:] = vectorizer(preprocessor(msg),model)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features,messages.Clase)
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.7666666666666667

In [ ]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(features,messages.Clase)

LogisticRegression()

In [ ]:
print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
print(predict_class('Mi nombre es'))
print(predict_class('Hola buenos'))
print(predict_class('¿puedo retirar un giro de dinero?'))
print(predict_class('Puede otra persona retirar mi pedido'))

['información']
['información']


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: ignored

### 2.2.2 ELMO

In [ ]:
features_elmo = []
for doc in features_raw:
  features_elmo.append(doc.mean(0))
features_elmo = np.vstack(features_elmo)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=1000),features_elmo,messages.Clase)
cv_results["test_score"].mean()

In [ ]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(features_elmo,messages.Clase)

LogisticRegression()

In [ ]:
print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
print(predict_class('Hola buen día como estas'))
print(predict_class('Mi nombre es Paula'))
print(predict_class('¿puedo retirar un giro de dinero?'))
print(predict_class('Puede otra persona retirar mi pedido'))

ValueError: ignored

## 2.3 Si el texto de entrada es del tipo "información”, busque ahora la pregunta mas similar y retorne la respuesta asociada.

## 2.4 Reporte el resultado con textos de prueba.

# 3. Análisis transformer 

Para el segundo chatbot utilice un transformer (por ejemplo BERT).

## 3.1 Utilice el modelo transformer para clasificar el texto de entrada, y para extraer la respuesta de la tabla de preguntas y respuestas cuando el mensaje sea del tipo “información”.

## 3.2 Reporte el tipo de red, y las métricas de entrenamiento usadas

## 3.3 Reporte el resultado con los textos de prueba.

# Conclusión